In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, cross_val_predict, train_test_split, KFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, balanced_accuracy_score, accuracy_score, mean_squared_error
from scipy.stats import pearsonr, ttest_rel
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE, Isomap
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import random
import re
from itertools import product
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
#proteomics architecture for classification
class proteomics_net(nn.Module):
    def __init__(self, input_size, proteomics_hidden_layers, output_size, dropout=0.1):
        super(proteomics_net, self).__init__()
            
        self.proteomics_layers = nn.Sequential(
            nn.Linear(input_size, proteomics_hidden_layers[0]),
            nn.ReLU(),
            nn.Linear(proteomics_hidden_layers[0], output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.proteomics_layers(x)
        return x

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)
        

In [ ]:
#GRU architecture for classification
class GRUNet(nn.Module):
    def __init__(self, input_size_codes, hidden_size, prediction_module_hidden_sizes, num_layers, output_size, dropout=0.1):
        super(GRUNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size_codes, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        self.prediction_module = nn.Sequential(
            nn.Linear(prediction_module_hidden_sizes[0], output_size),
            nn.Sigmoid()
        )



    def forward(self, x, lengths, interpretability=False):
        device = x.device
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.gru(x_packed, h0)
        out, _ = pad_packed_sequence(out_packed, batch_first=True)
        
        out = out[torch.arange(x.size(0)), lengths-1, :]
        out_final = self.prediction_module(out)
        if interpretability == False:
            return out_final
        else:
            return out_final, out

# Prepare the dataset
class PatientDataset(Dataset):
    def __init__(self, data, labels, lengths):
        self.data = data
        self.labels = labels
        self.lengths = lengths

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.lengths[idx]

# Custom collate function for DataLoader
def collate_fn(batch):
    data, labels, lengths = zip(*batch)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.float32)
    lengths = torch.tensor(lengths, dtype=torch.long)
    return data, labels, lengths


def normalize_dataset(dataset):
    for i, sequence in enumerate(dataset):
        dataset[i] = (sequence - sequence.mean(dim=0, keepdim=True)) / (sequence.std(dim=0, keepdim=True) + 1e-8)
    return dataset

def impute_missing_values(dataset):
    # Stack all tensors in the dataset along a new dimension, creating a tensor of shape (num_samples, max_seq_length, num_features)
    stacked_data = torch.stack(dataset)

    # Calculate the mean of each feature across all samples and sequences, ignoring NaN values
    feature_means = torch.nanmean(stacked_data, dim=(0, 1))

    # Iterate through the dataset (list of tensors)
    for i, sequence in enumerate(dataset):
        # Create a boolean mask indicating the positions of NaN values in the sequence
        mask = torch.isnan(sequence)

        # Replace NaN values in the sequence with the corresponding feature means
        # 'expand_as' is used to match the dimensions of the mask and the sequence
        dataset[i][mask] = feature_means.expand_as(sequence)[mask]

    return dataset

# Create DataLoaders
def create_dataloaders(patient_data, patient_outcomes, lengths, batch_size=64, normalize=False):
    
    X_train = impute_missing_values(patient_data)
    y_train = patient_outcomes
    
    if normalize:
        X_train = normalize_dataset(X_train)
        
    train_dataset = PatientDataset(X_train, y_train, lengths)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, worker_init_fn=worker_init_fn)
    return train_loader

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)


In [ ]:
# Full COMET architecture for classification
class joint_model(nn.Module):
    def __init__(self, input_size_codes, hidden_size, prediction_module_hidden_sizes, num_layers, output_size, input_size_proteomics, proteomics_hidden_layers, combined_hidden_layers, dropout=0.1):
        super(joint_model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size_codes, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        self.prediction_module = nn.Sequential(
            nn.Linear(prediction_module_hidden_sizes[0], output_size),
        )

        self.skip_connect_prot = nn.Sequential(
            nn.Linear(input_size_proteomics, proteomics_hidden_layers[0]),
            nn.ReLU(),
            nn.Linear(proteomics_hidden_layers[0], output_size),
            nn.Sigmoid()
        )
        
        self.combined_layers = nn.Sequential(
            nn.Linear(input_size_proteomics + hidden_size, output_size)
        )
        
        self.final_combine = nn.Linear(3, 1, bias=False)


    def forward(self, x, x_proteomics, lengths, interpretability=False, better_latent=None, better_ratio=0.5):
        device = x.device
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.gru(x_packed, h0)
        out, _ = pad_packed_sequence(out_packed, batch_first=True)
        
        out_ehr = out[torch.arange(x.size(0)), lengths-1, :]
        
        if x_proteomics == None:
            return out_ehr

        if better_latent is not None:
            out_ehr = better_ratio * better_latent + (1-better_ratio) * out_ehr
                
        out_combined = torch.cat((out_ehr, x_proteomics), 1)
        
        out_combined = self.combined_layers(out_combined)
        
        pred_proteomics = self.skip_connect_prot(x_proteomics)
        pred_ehr = self.prediction_module(out_ehr)
        
        final_pred = torch.sigmoid(self.final_combine(torch.cat((pred_proteomics, pred_ehr, out_combined), 1)))
#         final_pred = self.final_combine(torch.cat((pred_proteomics, pred_ehr), 1))
        
        if interpretability == False:
            return final_pred
        else:
            return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
    
    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [ ]:
class DataBuilder(Dataset):
    def __init__(self, x, y, standardizer):
        self.x, self.y, self.standardizer = x, y, standardizer
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return (self.x[index], self.y[index])
    def __len__(self):
        return self.len

In [ ]:
def run_experiment(EHR_codes, proteomics, patient_indices, outcomes, lengths, experiment_name, lr, lr_decay,
                   bs, prot_hidden_dim=32, train_indices=None, val_indices=None, test_indices=None, feature_types='EHR', model_path='', fine_tune=False, seed=42, num_layers=2,hidden_dim=400,
                   dropout=0.4, return_preds=False, return_interpretability=False, return_grads=False,
                   hyperparam_tuning=False):
    """
    EHR_codes: pre-processed data for codes of shape (num_patients, max_length, embedding_dim)v
    EHR_vitals: pre-processed data for codes of shape (num_patients, max_length, embedding_dim)
    proteomics: dataframe with proteomics data
    patient_indices: dataframe with sample IDs and row numbers in pre-processed matrices
    outcomes: array with DOS
    lengths: array with lengths (i.e. number of visits) to help with padding
    experiment_name: string for file name for models
    lr: float for learning rate
    lr_decay: float for learning rate decay
    bs: int for batch size
    feature_types: string either 'EHR', 'metab', 'both'
    model_path: string for file path to model if loading a pre-trained model
    fine_tune: boolean for whether or not EHR weight should be learned, can only be true if model != ''
    seed: int, random_seed for train/test/val split and seeding model 
    num_layers: number of GRU layers in RNN
    hidden_dim: hidden_dim of GRU output
    dropout: dropout weight in model
    return_preds: setting to control output of function, if True we return the predictions
    return_interpretability: setting to control output of function, if True we return some additional data to help with interpretability analysis
    return_grads: setting to control output of function, if True we return the gradient
    hyperparam_tuning: setting to control whether or not we save the model at each epoch (if True, we do not)
    """
    set_seed(seed)
    prediction_module_hidden_sizes = [hidden_dim,hidden_dim//2, hidden_dim//4, hidden_dim//8]
    
    assert feature_types in ['EHR','metab','both']   
    if feature_types == 'metab': assert model_path == ''
    if (model_path != '') & (feature_types == 'both'): assert fine_tune==True
    if hyperparam_tuning == False: assert train_indices == None

    
    if hyperparam_tuning == False:
        maternal_IDs = patient_indices['sample_ID']

        train_ratio = 0.70
        test_ratio = 0.15
        val_ratio = 0.15

        # First, split the unique_ids into train and temp (test + validation) sets
        train_ids, temp_ids = train_test_split(maternal_IDs, test_size=(test_ratio + val_ratio),random_state=seed, stratify=outcomes)
        
        patient_indices_temp = patient_indices.copy(deep=True)

        patient_indices_temp.index = patient_indices_temp['sample_ID']
        temp_indices = patient_indices_temp.loc[temp_ids,'array_index'].values

        # Next, split the temp_ids into test and validation sets
        test_ids, val_ids = train_test_split(temp_ids, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=seed, stratify=outcomes[temp_indices])
        proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_ids)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_ids)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_ids)]['array_index'].values

    #divide the input data into the train/test/val splits determined above 
    train_EHR_codes = EHR_codes[train_indices,:,:]
    if feature_types != 'EHR':
        train_proteomics = proteomics[train_indices,:]
        scaler = StandardScaler()
        train_proteomics = scaler.fit_transform(train_proteomics)
    train_outcomes = outcomes[train_indices]
    train_lengths = lengths[train_indices]
    
    test_EHR_codes = EHR_codes[test_indices, :, :]
    if feature_types != 'EHR':
        test_proteomics = proteomics[test_indices, :]
        scaler = StandardScaler()
        test_proteomics = scaler.fit_transform(test_proteomics)
    test_outcomes = outcomes[test_indices]
    test_lengths = lengths[test_indices]

    val_EHR_codes = EHR_codes[val_indices, :, :]
    if feature_types != 'EHR':
        val_proteomics = proteomics[val_indices, :]
        scaler = StandardScaler()
        val_proteomics = scaler.fit_transform(val_proteomics)
    val_outcomes = outcomes[val_indices]
    val_lengths = lengths[val_indices]
        
    all_EHR_codes = EHR_codes
    scaler = StandardScaler()
    all_outcomes = outcomes
    all_lengths = lengths
    
    #convert to tensors for input to pytorch models
    train_EHR_codes = [torch.tensor(data).float() for data in train_EHR_codes]  
    train_EHR_codes = [torch.nan_to_num(x) for x in train_EHR_codes]
    if feature_types != 'EHR':
        train_proteomics = torch.tensor(train_proteomics).float()
        train_proteomics = torch.nan_to_num(train_proteomics)
    train_outcomes = torch.tensor(train_outcomes).float()
    
    test_EHR_codes = [torch.tensor(data).float() for data in test_EHR_codes]
    test_EHR_codes = [torch.nan_to_num(x) for x in test_EHR_codes]
    if feature_types != 'EHR':
        test_proteomics = torch.tensor(test_proteomics).float()
        test_proteomics = torch.nan_to_num(test_proteomics)
    test_outcomes = torch.tensor(test_outcomes).float()

    val_EHR_codes = [torch.tensor(data).float() for data in val_EHR_codes]
    val_EHR_codes = [torch.nan_to_num(x) for x in val_EHR_codes]
    if feature_types != 'EHR':
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
    val_outcomes = torch.tensor(val_outcomes).float()
    
    all_EHR_codes = [torch.tensor(data).float() for data in all_EHR_codes]
    all_EHR_codes = [torch.nan_to_num(x) for x in all_EHR_codes]
    all_outcomes = torch.tensor(all_outcomes).float()

    #create necessary data loaders
    if feature_types != 'EHR':
        data_set_train = DataBuilder(train_proteomics, train_outcomes, scaler)
        train_loader_proteomics = DataLoader(dataset=data_set_train,batch_size=bs, worker_init_fn=worker_init_fn)
    train_loader_codes = create_dataloaders(train_EHR_codes, train_outcomes, train_lengths, bs)

    if feature_types != 'EHR':
        data_set_test = DataBuilder(test_proteomics, test_outcomes, scaler)
        test_loader_proteomics = DataLoader(dataset=data_set_test,batch_size=bs, worker_init_fn=worker_init_fn)
    test_loader_codes = create_dataloaders(test_EHR_codes, test_outcomes, test_lengths, bs)

    if feature_types != 'EHR':
        data_set_val = DataBuilder(val_proteomics, val_outcomes, scaler)
        val_loader_proteomics = DataLoader(dataset=data_set_val,batch_size=100*bs, worker_init_fn=worker_init_fn)
    val_loader_codes = create_dataloaders(val_EHR_codes, val_outcomes, val_lengths, 100*bs)
            
    all_loader_codes = create_dataloaders(all_EHR_codes, all_outcomes, all_lengths, 1000)   
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #initialize correct model, if we're doing baseline experiments
    if model_path == '':
        if feature_types == 'EHR':
            model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout).to(device)
        elif feature_types == 'metab':
            model = proteomics_net(proteomics.shape[1], [prot_hidden_dim], 1, dropout).to(device)
        elif feature_types == 'both':
            model = joint_model(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, proteomics.shape[1], [prot_hidden_dim], [64, 32, 16, 8], dropout).to(device)
    
    #initialize correct model if we're using pre-trained EHR weight from an existing model
    else:
        if feature_types == 'EHR':
            model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout)
            model_state_dict = torch.load(model_path)
            model.load_state_dict(model_state_dict)
            model.to(device)
            if fine_tune == False:
                model.eval()
                criterion = nn.BCELoss()
                val_predictions = []
                val_true_labels = []
                running_loss_val, num_samples_val = 0, 0
                with torch.no_grad():
                    for (inputs_codes, labels_codes, lengths_codes) in (val_loader_codes):
                            inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                            outputs = model(inputs_codes, lengths_codes)

                            loss = criterion(outputs.squeeze(), labels)
                            running_loss_val += (loss.item()*lengths_codes.shape[0])
                            num_samples_val += lengths_codes.shape[0]
                            val_predictions.extend(outputs.squeeze().tolist())
                            val_true_labels.extend(labels.tolist())
                    
                val_loss = running_loss_val / (num_samples_val)
                pearson_corr = roc_auc_score(val_true_labels, val_predictions)

                print(f'Total Loss: {val_loss:.4f}, AUC: {pearson_corr:.4f}')
                if return_preds == True:
                    return pearson_corr, val_loss, None, val_true_labels, val_predictions, val_indices
                else:
                    return pearson_corr, val_loss, None
            elif fine_tune == True:
                model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout)
                model_state_dict = torch.load(model_path)
                model.load_state_dict(model_state_dict, strict=False)
                model.to(device)
                for name, param in model.named_parameters():
                    if ('gru' in name):
                        param.requires_grad = False

        elif feature_types == 'both':
            model = joint_model(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, proteomics.shape[1], [prot_hidden_dim], [64, 32, 16, 8], dropout).to(device)
            model_state_dict = torch.load(model_path)
            gru_weights = {}
            for k,v in zip(model_state_dict.keys(), model_state_dict.values()):
                if ('gru' in k) | ('pred' in k):
                    gru_weights[k] = v
            model.load_state_dict(gru_weights, strict=False)
            model.to(device)
            
            for name, param in model.named_parameters():
                if ('gru' in name):
                    param.requires_grad = False

    #prepare for training    
    epoch_arr = []
    num_samples_in_batch = []
    gradient_arr = []
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr_decay)
    num_epochs = 200
    train_losses = []
    test_losses = []
    val_losses = []
    best_loss = np.inf

    for epoch in tqdm(range(num_epochs)):
        model.train()
        if hyperparam_tuning == False:
            torch.cuda.synchronize()  # Wait for all CUDA kernels to finish
            torch.save(model.state_dict(), './{}_epoch{}.pth'.format(experiment_name, epoch))
        running_loss_train, num_train_samples = 0, 0
        if feature_types == 'EHR':
            for (inputs_codes, labels_codes, lengths_codes)in train_loader_codes:
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)

                optimizer.zero_grad()
                
                outputs = model(inputs_codes, lengths_codes)
                if outputs.shape[0] == 1:
                    loss = criterion(outputs.squeeze().unsqueeze(0), labels)
                else:
                    loss = criterion(outputs.squeeze(), labels)

                loss.backward()
                optimizer.step()
                running_loss_train += (loss.item()*lengths_codes.shape[0])
                num_train_samples += lengths_codes.shape[0]
        else:
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(train_loader_codes, train_loader_proteomics)):
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                optimizer.zero_grad()
                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)

                if outputs.shape[0] == 1:
                    loss = criterion(outputs.squeeze().unsqueeze(0), labels)
                else:
                    loss = criterion(outputs.squeeze(), labels)
                
                loss.backward()
                optimizer.step()
                if return_grads:
                    epoch_arr.append(epoch)
                    num_samples_in_batch.append(outputs.shape[0])
                    gradient_arr.append(model.skip_connect_prot[0].weight.grad.cpu().numpy()[0])
                running_loss_train += (loss.item()*lengths_codes.shape[0])
                num_train_samples += lengths_codes.shape[0]

        train_loss = running_loss_train / num_train_samples
        train_losses.append(train_loss)

        model.eval()
        running_loss_test, num_samples_test = 0, 0
        predictions = []
        true_labels = []
        if feature_types == 'EHR':
            with torch.no_grad():
                for (inputs_codes, labels_codes, lengths_codes)in test_loader_codes:
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    outputs = model(inputs_codes, lengths_codes)
                    outputs = outputs.squeeze(1)
                    loss = criterion(outputs, labels)
                    running_loss_test += (loss.item()*lengths_codes.shape[0])
                    num_samples_test += lengths_codes.shape[0]
                    predictions.extend(outputs.tolist())
                    true_labels.extend(labels.tolist())
        else:
            with torch.no_grad():
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(test_loader_codes, test_loader_proteomics)):
                    if feature_types != 'metab':
                        inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    if feature_types != 'EHR':
                        inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                    if feature_types == 'metab':
                        outputs = model(inputs_proteomics)
                    elif feature_types == 'EHR':
                        outputs = model(inputs_codes, lengths_codes)
                    elif feature_types == 'both':
                        outputs = model(inputs_codes, inputs_proteomics, lengths_codes)
                    
                    outputs = outputs.squeeze(1)
                    loss = criterion(outputs, labels)
                    
                    running_loss_test += (loss.item()*lengths_codes.shape[0])
                    num_samples_test += lengths_codes.shape[0]
                    predictions.extend(outputs.tolist())
                    true_labels.extend(labels.tolist())
        test_loss = running_loss_test / (num_samples_test)
        test_losses.append(test_loss)
        
        #check for early stopping on test set
        patience = 5
        pearson_corr = roc_auc_score(true_labels, predictions)

        #print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, AUC: {pearson_corr:.4f}')
        
        if test_loss < best_loss:
            best_loss = test_loss
            counter = 0
            torch.cuda.synchronize()  # Wait for all CUDA kernels to finish
            torch.save(model.state_dict(), './{}.pth'.format(experiment_name))
        else:
            counter += 1
            if counter >= patience:
                break

    #once training is finished, make predictions on validation set
    model.load_state_dict(torch.load('./{}.pth'.format(experiment_name)))
    running_loss_val, num_samples_val = 0, 0
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        model.eval()
        if feature_types == 'EHR':
            for (inputs_codes, labels_codes, lengths_codes) in val_loader_codes:
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)
                    
                loss = criterion(outputs.squeeze(), labels)
                running_loss_val += (loss.item()*lengths_codes.shape[0])
                num_samples_val += lengths_codes.shape[0]
                val_predictions.extend(outputs.squeeze().tolist())
                val_true_labels.extend(labels.tolist())
        else:
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(val_loader_codes, val_loader_proteomics)):
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)

                loss = criterion(outputs.squeeze(), labels)
                running_loss_val += (loss.item()*lengths_codes.shape[0])
                num_samples_val += lengths_codes.shape[0]
                val_predictions.extend(outputs.squeeze().tolist())
                val_true_labels.extend(labels.tolist())
    val_loss = running_loss_val / (num_samples_val)
    val_losses.append(val_loss)
    pearson_corr = roc_auc_score(val_true_labels, val_predictions)

    print(f'Epoch: {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, AUC: {pearson_corr:.4f}')
    
    if hyperparam_tuning == True:
        os.remove('./{}.pth'.format(experiment_name))
    if return_preds == True:
        if return_interpretability == False:
            if return_grads:
                df = pd.DataFrame([epoch_arr, num_samples_in_batch, gradient_arr]).T
                df.columns = ['epoch', 'num_samples_in_batch','gradient']
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, df, train_losses, test_losses, val_losses, train_indices, test_indices
            else:
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices
        else:
            if return_grads:
                df = pd.DataFrame([epoch_arr, num_samples_in_batch, gradient_arr]).T
                df.columns = ['epoch', 'num_samples_in_batch','gradient']
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, interpretability_outputs, None, None, df, train_losses, test_losses, val_losses, train_indices, test_indices
            else:
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, interpretability_outputs, None, None
    else:
        return pearson_corr, val_loss, None



In [ ]:
#load omics cohort data w/ omics word2vec model
RNN_data_codes_omics = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/RNN_data_omics_omicsw2v_lc.npy')
RNN_data_outcomes_omics = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/outcomes_omics_omicsw2v_lc_3yr.npy')
RNN_data_lengths_omics = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/visit_count_omics_omicsw2v_lc.npy')
patient_indices_omics = pd.read_csv('/mnt/project/cancer_mortality_3yr_LARGER_data/eid_indices_omics_omicsw2v_lc.csv').drop('Unnamed: 0',axis=1)
patient_indices_omics.columns = ['sample_ID','array_index']
RNN_data_codes_omics.shape, RNN_data_outcomes_omics.shape, RNN_data_lengths_omics.shape, patient_indices_omics.shape

In [ ]:
#loads omics cohort data w/ embeddings from pre-trained word2vec model (for COMET experiments)
RNN_data_codes = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/RNN_data_omics_PTw2v_lc.npy')
RNN_data_outcomes = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/outcomes_omics_PTw2v_lc_3yr.npy')
RNN_data_lengths = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/visit_count_omics_PTw2v_lc.npy')
patient_indices = pd.read_csv('/mnt/project/cancer_mortality_3yr_LARGER_data/eid_indices_omics_PTw2v_lc.csv').drop('Unnamed: 0',axis=1)
patient_indices.columns = ['sample_ID','array_index']
RNN_data_codes.shape, RNN_data_outcomes.shape, RNN_data_lengths.shape, patient_indices.shape

In [ ]:
RNN_data_codes_PTMODEL = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/RNN_data_PT_lc_LARGER.npy')
RNN_data_outcomes_PTMODEL = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/outcomes_PT_lc_LARGER.npy')
RNN_data_lengths_PTMODEL = np.load('/mnt/project/cancer_mortality_3yr_LARGER_data/visit_count_PT_lc_LARGER.npy')
patient_indices_PTMODEL = pd.read_csv('/mnt/project/cancer_mortality_3yr_LARGER_data/eid_indices_PT_lc_LARGER.csv').drop('Unnamed: 0',axis=1)
patient_indices_PTMODEL.columns = ['sample_ID','array_index']
RNN_data_codes_PTMODEL.shape, RNN_data_outcomes_PTMODEL.shape, RNN_data_lengths_PTMODEL.shape, patient_indices_PTMODEL.shape

In [ ]:
overall_best_params = {}

In [ ]:
outcome_list = ['mortality']

In [ ]:
NUM_TRIALS = 25

In [ ]:
%%time
#load all proteomics
proteomics = pd.read_csv('/mnt/project/all_proteomics_lc.csv')
proteomics = proteomics[proteomics['eid'].isin(patient_indices['sample_ID'])]
proteomics = proteomics.dropna(axis=1, how='all')
proteomics.shape

In [ ]:
proteomics = proteomics.rename({'eid':'sample_ID'},axis=1)

In [ ]:
#impute missing values
min_values = proteomics.drop(columns=['sample_ID']).min()

impute_values = min_values / 2

# Replace NaN values with the impute values calculated for each protein
for column in proteomics.columns:
    if column != 'sample_ID':
        proteomics[column].fillna(impute_values[column], inplace=True)

### Run cell below if loading old hyperparams

In [ ]:
#if we're using the cell for downstream analysis and skipping the hyperparam selection, run this cell to load previously saved hyperparams
overall_best_params = pickle.load(open('/mnt/project/cancer_mortality_3yr_LARGER_models_results/best_hyperparams_LARGER.pkl','rb'))

best_num_layers = overall_best_params['mortality']['PT']['num_layers']
best_dropout = overall_best_params['mortality']['PT']['dropout']
best_model_name = overall_best_params['mortality']['PT']['model_name']
best_hidden_dim = overall_best_params['mortality']['PT']['hidden_dim']
print(best_num_layers, best_dropout, best_hidden_dim, best_model_name)

best_num_layers_omics = overall_best_params['mortality']['exp1']['num_layers']
best_dropout_omics = overall_best_params['mortality']['exp1']['dropout']
best_hidden_dim_omics = overall_best_params['mortality']['exp1']['hidden_dim']
print(best_num_layers_omics, best_dropout_omics, best_hidden_dim_omics)

best_protein_hidden = overall_best_params['mortality']['exp2']['prot_hidden_dim']
print(best_protein_hidden)

### Hyperparam Tuning

In [ ]:
# Hyperparameter grid
grid_search = {
    'batch_size': [512],
    'lr': [1e-2, 1e-3, 1e-4],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
    'layers': [2, 4],
    'hidden_dim': [400]
}

# Generate all combinations of hyperparameters
all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

#run hyperparam selection for PT model
num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i] = {'PT': {}}

    maternal_IDs = patient_indices_PTMODEL['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(maternal_IDs):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        

        train_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = param_set['layers']
            hidden_dim = param_set['hidden_dim']
            
            model_name = 'PT_MODEL_{}_{}_{}_{}_{}'.format(layers,lr,lr_decay,dropout, hidden_dim)
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_PTMODEL, None,
            patient_indices_PTMODEL, RNN_data_outcomes_PTMODEL, RNN_data_lengths_PTMODEL, model_name, 
            lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                                                       val_indices=val_indices,feature_types='EHR', model_path='', fine_tune=False, seed=42, 
            hidden_dim=hidden_dim, num_layers=layers, dropout=dropout, hyperparam_tuning=True)
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)
        print('PT')
        print('outcome {}'.format(i))

In [ ]:
#save best hyperparams
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
model_name = 'PT_MODEL_{}_{}_{}_{}_{}'.format(num_layers,lr,lr_decay,dropout, hidden_dim)
overall_best_params['mortality']['PT'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs), 'model_name': model_name}
with open("best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
best_num_layers = overall_best_params['mortality']['PT']['num_layers']
best_dropout = overall_best_params['mortality']['PT']['dropout']
best_model_name = overall_best_params['mortality']['PT']['model_name']
best_hidden_dim = overall_best_params['mortality']['PT']['hidden_dim']
best_num_layers, best_dropout, best_hidden_dim, best_model_name

In [ ]:
hyperparam_df.to_csv('hyperparam_sweep_PT_LARGER.csv')


In [ ]:
%%bash
dx upload hyperparam_sweep_PT_LARGER.csv --path /


In [ ]:
overall_best_params

In [ ]:
%%time
#train pre-trained model!
val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_PTMODEL, proteomics,
    patient_indices_PTMODEL, RNN_data_outcomes_PTMODEL, RNN_data_lengths_PTMODEL, overall_best_params['mortality']['PT']['model_name'], 
    overall_best_params['mortality']['PT']['lr'], overall_best_params['mortality']['PT']['lr_decay'],
       512, feature_types='EHR', model_path='', fine_tune=False, seed=3, 
    hidden_dim=overall_best_params['mortality']['PT']['hidden_dim'], num_layers=overall_best_params['mortality']['PT']['num_layers'], dropout=overall_best_params['mortality']['PT']['dropout'], hyperparam_tuning=False)


In [ ]:
grid_search = {
    'batch_size': [16],
    'lr': [1e-2, 1e-1, 1e-3, 1e-4],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
    'layers': [2, 4],
    'hidden_dim': [400]
}
all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
#run hyperparam loop for omics baseline
num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp1'] = {}

    IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, RNN_data_outcomes_omics):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = param_set['layers']
            hidden_dim = param_set['hidden_dim']
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                                                       patient_indices_omics, RNN_data_outcomes_omics,
                                                       RNN_data_lengths_omics, 'EHR_omics_only',
                                                       lr, lr_decay, bs, 
                                                       train_indices=train_indices, test_indices=test_indices,
                                                       val_indices=val_indices, feature_types='EHR', model_path='',
                                                       fine_tune=False, seed=42, hidden_dim=hidden_dim,
                                                       num_layers=layers, dropout=dropout, hyperparam_tuning=True)

            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 1')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))

print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['mortality']['exp1'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
overall_best_params

In [ ]:
best_num_layers_omics = overall_best_params['mortality']['exp1']['num_layers']
best_dropout_omics = overall_best_params['mortality']['exp1']['dropout']
best_hidden_dim_omics = overall_best_params['mortality']['exp1']['hidden_dim']
best_num_layers_omics, best_dropout_omics, best_hidden_dim_omics

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
              'prot_hidden_dim': [16,32,64]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
#run hyperparam loop for EHR baseline
num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
phd_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp2'] = {}

    IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, RNN_data_outcomes_omics):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout_omics
            lr_decay = param_set['lr_decay']
            prot_hidden_dim = param_set['prot_hidden_dim']
            layers = best_num_layers_omics
            hidden_dim = best_hidden_dim_omics
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'proteomics_omics_only', 
                lr, lr_decay, bs, prot_hidden_dim, train_indices=train_indices, test_indices=test_indices, val_indices=val_indices,
                feature_types='metab', model_path='', fine_tune=False, seed=42, hidden_dim=hidden_dim,
                num_layers=layers, dropout=dropout, hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)
            phd_arr.append(prot_hidden_dim)

        print('experiment 2')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, phd_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd)

overall_best_params['mortality']['exp2'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs), 'prot_hidden_dim':int(phd)}
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
best_protein_hidden = overall_best_params['mortality']['exp2']['prot_hidden_dim']

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
              'prot_hidden_dim': [best_protein_hidden]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
#run hyperparam loop for joint data baseline
num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
phd_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp3'] = {}

    IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, RNN_data_outcomes_omics):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            prot_hidden_dim = param_set['prot_hidden_dim']
            layers = best_num_layers_omics
            hidden_dim = best_hidden_dim_omics
            print(param_set)
            
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'both_omics_only', 
                lr, lr_decay, bs, prot_hidden_dim, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices, feature_types='both', model_path='', fine_tune=False, seed=42,
                hidden_dim=hidden_dim, num_layers=layers, dropout=dropout, hyperparam_tuning=True)

            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)
            phd_arr.append(prot_hidden_dim)

        print('experiment 3')
        print('outcome {}'.format(i))


In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, phd_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd)

overall_best_params['mortality']['exp3'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs), 'prot_hidden_dim':int(phd)}
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
#test PT model on omics population, hyperparams are saved to avoid downstream issues but there is no training done here so hyperparams don't matter
for i in tqdm(outcome_list):
    overall_best_params[i]['exp4'] = {}
    results_dict = {}

    # choose large enough bs to do inference in one batch
    bs = 1000
    lr = 0.1
    dropout = best_dropout
    lr_decay = 0.1
    layers = best_num_layers
    hidden_dim = best_hidden_dim
    # print(param_set)
    val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, proteomics,
            patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_omics_PT', 
            lr, lr_decay, bs, feature_types='EHR', model_path='./{}.pth'.format(best_model_name),
                                               fine_tune=False, seed=42,
                                               hidden_dim=hidden_dim,
                                              num_layers=layers, dropout=dropout, hyperparam_tuning=False)

    results_dict[val_loss] = {'num_layers': layers,'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                            'hidden_dim': hidden_dim, 'batch_size': bs}

    print('experiment 4')
    print('outcome {}'.format(i))
    overall_best_params[i]['exp4'] = results_dict[min(results_dict.keys())]
    print(results_dict[min(results_dict.keys())])

# Save the best hyperparameters to a pickle file
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
    pickle.dump(overall_best_params, f)

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
               'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
#run hyperparam loop for fine tuning pre-trained EHR model

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp5'] = {}

    IDs = patient_indices['sample_ID'].unique()
    
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, RNN_data_outcomes):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout
            lr_decay = param_set['lr_decay']
            layers = best_num_layers
            hidden_dim = best_hidden_dim
            print(param_set)
            
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, input_proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_omics_PT_FT',
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices, feature_types='EHR',
                model_path='./{}.pth'.format(best_model_name),
                fine_tune=True, seed=42, hidden_dim=hidden_dim, num_layers=layers, dropout=dropout,
                                                       hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 5')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['mortality']['exp5'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
overall_best_params

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
              'prot_hidden_dim':[best_protein_hidden]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
#run hyperparam loop for COMET models

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
phd_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp6'] = {}

    IDs = patient_indices['sample_ID'].unique()
    
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, RNN_data_outcomes):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout
            lr_decay = param_set['lr_decay']
            prot_hidden_dim = param_set['prot_hidden_dim']
            layers = best_num_layers
            hidden_dim = best_hidden_dim
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, input_proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'both_PT', 
                lr, lr_decay, bs, prot_hidden_dim, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices,feature_types='both',
                model_path='/mnt/project/cancer_mortality_3yr_LARGER_models_results/{}.pth'.format(best_model_name),
                fine_tune=True, seed=42, hidden_dim=hidden_dim, num_layers=layers, dropout=dropout,
                                                        hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)
            phd_arr.append(prot_hidden_dim)

        print('experiment 6')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, phd_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs','phd']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd = hyperparam_df['val_loss'].idxmin()
#num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd = hyperparam_df.sort_values('val_loss').index[3]
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs, phd)

overall_best_params['mortality']['exp6'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs),'prot_hidden_dim':int(phd)}
with open("./best_hyperparams_LARGER.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
overall_best_params

In [ ]:
%%bash
dx upload best_hyperparams_LARGER.pkl --path /

In [ ]:
best_model_name

In [ ]:
5

In [ ]:
%%bash
dx upload ./PT_MODEL_2.0_0.001_0.001_0.1_400.0.pth --path /


### Run cell below if skipping modeling

In [ ]:
#load existing modeling results if we're not doing modeling and only want to do downstream analysis
results = pickle.load(open('/mnt/project/cancer_mortality_3yr_LARGER_models_results/results_3yr_mortality_LARGER.pkl','rb'))

### Modeling

In [ ]:
#experiment 1 = baseline model EHR features
#experiment 2 = baseline model metab features
#experiment 3 = baseline model all features
#experiment 4 = only pretrained model
#experiment 5 = fine tune pretrained model 
#experiment 6 = full COMET

num_iterations = 25
results = {}
for i in tqdm(outcome_list):
    results[i] = {'exp1':[],'exp2':[],'exp3':[],'exp4':[],'exp5':[],'exp6':[]}
    for j in tqdm(range(num_iterations)):
        print('experiment 1')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'EHR_omics_only_{}'.format(j), 
                overall_best_params[i]['exp1']['lr'], overall_best_params[i]['exp1']['lr_decay'],
                overall_best_params[i]['exp1']['batch_size'], feature_types='EHR', model_path='', fine_tune=False, seed=j,
                                hidden_dim=overall_best_params[i]['exp1']['hidden_dim'],
                                 num_layers=overall_best_params[i]['exp1']['num_layers'],
                                 dropout=overall_best_params[i]['exp1']['dropout'], return_preds=True)
        results[i]['exp1'].append(val_auc)
        
        print('experiment 2')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'proteomics_omics_only_{}'.format(j), 
                overall_best_params[i]['exp2']['lr'], overall_best_params[i]['exp2']['lr_decay'],
                overall_best_params[i]['exp2']['batch_size'], overall_best_params[i]['exp2']['prot_hidden_dim'],
                                 feature_types='metab', model_path='', fine_tune=False, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers, dropout=best_dropout, return_preds=True)
        results[i]['exp2'].append(val_auc)
        
        print('experiment 3')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'both_omics_only_{}'.format(j), 
                overall_best_params[i]['exp3']['lr'], overall_best_params[i]['exp3']['lr_decay'],
                overall_best_params[i]['exp3']['batch_size'], overall_best_params[i]['exp3']['prot_hidden_dim'],
                                 feature_types='both', model_path='', fine_tune=False, seed=j,
                                hidden_dim=overall_best_params[i]['exp3']['hidden_dim'],
                                 num_layers=overall_best_params[i]['exp3']['num_layers'],
                                 dropout=overall_best_params[i]['exp3']['dropout'],
                                 return_preds=True, return_interpretability=True, return_grads=True)
        results[i]['exp3'].append(val_auc)
        
        print('experiment 4')
        val_auc = run_experiment(RNN_data_codes, proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_PT_{}'.format(j), 
                overall_best_params[i]['exp4']['lr'], overall_best_params[i]['exp4']['lr_decay'],
                overall_best_params[i]['exp4']['batch_size'], feature_types='EHR',
                                 model_path='/mnt/project/cancer_mortality_3yr_LARGER_models_results/{}.pth'.format(best_model_name),
                                 fine_tune=False, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout, return_preds=True)
        results[i]['exp4'].append(val_auc)

        print('experiment 5')
        val_auc = run_experiment(RNN_data_codes, proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_PT_FT_{}'.format(j), 
                overall_best_params[i]['exp5']['lr'], overall_best_params[i]['exp5']['lr_decay'],
                overall_best_params[i]['exp5']['batch_size'], feature_types='EHR',
                                 model_path='/mnt/project/cancer_mortality_3yr_LARGER_models_results/{}.pth'.format(best_model_name),
                                 fine_tune=True, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout, return_preds=True)
        results[i]['exp5'].append(val_auc)

        print('experiment 6')
        val_auc = run_experiment(RNN_data_codes, proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'both_PT_FT_{}'.format(j), 
                overall_best_params[i]['exp6']['lr'], overall_best_params[i]['exp6']['lr_decay'],
                overall_best_params[i]['exp6']['batch_size'], overall_best_params[i]['exp6']['prot_hidden_dim'],
                                 feature_types='both',
                                model_path='/mnt/project/cancer_mortality_3yr_LARGER_models_results/{}.pth'.format(best_model_name),
                                 fine_tune=True, seed=j,
                                 hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout,
                                 return_preds=True, return_interpretability=True, return_grads=True)
        results[i]['exp6'].append(val_auc)
    

### Compute performance metrics from validation set

In [ ]:
for exp in results['mortality'].keys():
    true_outcomes = []
    total_preds = []
    indices = []
    for i in results['mortality'][exp]:
        true_outcomes.extend(i[3])
        
        total_preds.extend(i[4])
        indices.extend(i[5])
    
        df = pd.DataFrame([true_outcomes,total_preds,indices]).T
        df.columns = ['true_outcome','pred','index']
        df = df.groupby('index').mean()
    print(exp)
    print('AUC: {}'.format(roc_auc_score(df['true_outcome'], df['pred'])))
    print('AUPRC: {}'.format(average_precision_score(df['true_outcome'], df['pred'])))
    epsilon = 1e-15
    df['pred'] = np.clip(df['pred'], epsilon, 1 - epsilon)

    # Calculating the BCELoss
    bce_loss = -np.mean(df['true_outcome'] * np.log(df['pred']) + (1 - df['true_outcome']) * np.log(1 - df['pred']))

    print(bce_loss)

In [ ]:
import os
import subprocess
from tqdm import tqdm

# Specify the directory containing the files you want to upload
directory_path = './'

# Specify the destination path on DNA Nexus
destination_path = './cancer_mortality_3yr_LARGER_models_results/'

# List all files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]


In [ ]:
progress_bar = tqdm(files, desc="Uploading files")

for file in progress_bar:
    # Construct the bash command for the current file
    command = f'dx upload "{os.path.join(directory_path, file)}" --path {destination_path}'
    
    # Update tqdm progress bar description
    progress_bar.set_description(f"Uploading {file}")
    
    # Execute the bash command (ignore the output)
    subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print("All files have been uploaded.")

In [ ]:
with open("./results_3yr_mortality_LARGER.pkl", "wb") as f:
        pickle.dump(results, f)

In [ ]:
%%bash
dx upload ./results_3yr_mortality_LARGER.pkl --path ./cancer_mortality_3yr_LARGER_models_results/


In [ ]:
%%bash
dx upload ./best_hyperparams_LARGER.pkl --path ./cancer_mortality_3yr_LARGER_models_results/


## Downstream Analysis

### Feature Importance: Integrated Gradients

In [ ]:
!pip install captum

In [ ]:
from captum.attr import IntegratedGradients

In [ ]:
from captum.attr import DeepLift
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
#5 for val data, 13 for train data
imp_index = 5
all_data = True


# Path to models
model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_PT_FT_{i}.pth' for i in range(25)]

# Initialize array to store feature importance
importance_proteomics_all = torch.zeros((len(model_paths), proteomics.shape[1]-1))
                                        
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, proteomics, codes, lengths):
        return self.model(codes, proteomics, lengths)
# Iterate over all models
# mean_proteomics_baseline = torch.mean(inputs_proteomics, dim=0, keepdim=True).to(device)

input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

for model_idx, model_path in tqdm(enumerate(model_paths),  total=len(model_paths)):
    
    
    model = joint_model(RNN_data_codes.shape[2], best_hidden_dim, [best_hidden_dim], best_num_layers, 1,
                    input_proteomics.shape[1], [best_protein_hidden], [], best_dropout).to(device)
    

    if all_data == False:
        val_proteomics = input_proteomics[results['mortality']['exp6'][model_idx][imp_index],:]
    else:
        val_proteomics = input_proteomics
    #This has to be done within the val set!
    scaler = StandardScaler()
    val_proteomics = scaler.fit_transform(val_proteomics)

    if all_data == False:
        EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes[results['mortality']['exp6'][model_idx][imp_index],:,:]]  
        EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
        val_outcomes = torch.tensor(RNN_data_outcomes[results['mortality']['exp6'][model_idx][imp_index]]).float()

        outcome_mean = torch.mean(val_outcomes)
        outcome_sd = torch.std(val_outcomes)

        data_set = DataBuilder(val_proteomics, val_outcomes, scaler)
        loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
        loader_codes = create_dataloaders(EHR_codes, val_outcomes, RNN_data_lengths[results['mortality']['exp6'][model_idx][imp_index]], 100000)
    else:
        EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
        EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
        val_outcomes = torch.tensor(RNN_data_outcomes).float()

        outcome_mean = torch.mean(val_outcomes)
        outcome_sd = torch.std(val_outcomes)

        data_set = DataBuilder(val_proteomics, val_outcomes, scaler)
        loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
        loader_codes = create_dataloaders(EHR_codes, val_outcomes, RNN_data_lengths, 100000)
    



    
    # Load the model
    model_state_dict = torch.load(model_path)
    model.load_state_dict(model_state_dict)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    model.eval()

    model_wrapper = ModelWrapper(model)
    ig = IntegratedGradients(model_wrapper)
    # Compute the feature importance for this model
    for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in zip(loader_codes, loader_proteomics):
        inputs_codes, labels_codes = inputs_codes.to(device), labels_codes.to(device)
        inputs_proteomics, labels_proteomics = inputs_proteomics.to(device), labels_proteomics.to(device)
        
        def forward_func(proteomics, codes, lengths):
            return model(codes, proteomics, lengths)
        
        # Compute feature importances using a custom forward function; CONSIDER THE BASELINE
        importance_proteomics = ig.attribute(inputs_proteomics, additional_forward_args=(inputs_codes, lengths_codes))
        # Store the feature importance for this model
        importance_proteomics_all[model_idx] = importance_proteomics.mean(dim=0).cpu().detach()
    
# Compute the average feature importance across all models
importance_proteomics_avg = torch.mean((importance_proteomics_all), dim=0)*1e11

# Create dataframe for feature importances
importance_df = pd.DataFrame([importance_proteomics_avg.numpy(), proteomics.drop(['sample_ID'],axis=1).columns]).T
importance_df.columns = ['importance_PT','name']
importance_df.sort_values('importance_PT')

In [ ]:
importance_df['abs_PT'] = np.abs(importance_df['importance_PT'])
importance_df.sort_values('abs_PT', ascending=False)


In [ ]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
#5 for val data, 13 for train data
imp_index = 5
all_data = True


# Path to models
model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_omics_only_{i}.pth' for i in range(25)]

# Initialize array to store feature importance
importance_proteomics_all = torch.zeros((len(model_paths), proteomics.shape[1]-1))
                                        
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, proteomics, codes, lengths):
        return self.model(codes, proteomics, lengths)
# Iterate over all models
# mean_proteomics_baseline = torch.mean(inputs_proteomics, dim=0, keepdim=True).to(device)

input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

for model_idx, model_path in tqdm(enumerate(model_paths), total=len(model_paths)):
    
    
    model = joint_model(RNN_data_codes_omics.shape[2], best_hidden_dim_omics, [best_hidden_dim_omics], best_num_layers_omics, 1,
                    input_proteomics.shape[1], [best_protein_hidden], [], best_dropout_omics).to(device)
    

    if all_data == False:
        val_proteomics = input_proteomics[results['mortality']['exp3'][model_idx][imp_index],:]
    else:
        val_proteomics = input_proteomics
    #This has to be done within the val set!
    scaler = StandardScaler()
    val_proteomics = scaler.fit_transform(val_proteomics)

    if all_data == False:
        EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes_omics[results['mortality']['exp3'][model_idx][imp_index],:,:]]  
        EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
        val_outcomes = torch.tensor(RNN_data_outcomes_omics[results['mortality']['exp3'][model_idx][imp_index]]).float()

        outcome_mean = torch.mean(val_outcomes)
        outcome_sd = torch.std(val_outcomes)

        data_set = DataBuilder(val_proteomics, val_outcomes, scaler)
        loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
        loader_codes = create_dataloaders(EHR_codes, val_outcomes, RNN_data_lengths_omics[results['mortality']['exp3'][model_idx][imp_index]], 100000)
    else:
        EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
        EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
        val_outcomes = torch.tensor(RNN_data_outcomes).float()

        outcome_mean = torch.mean(val_outcomes)
        outcome_sd = torch.std(val_outcomes)

        data_set = DataBuilder(val_proteomics, val_outcomes, scaler)
        loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
        loader_codes = create_dataloaders(EHR_codes, val_outcomes, RNN_data_lengths, 100000)
    



    
    # Load the model
    model_state_dict = torch.load(model_path)
    model.load_state_dict(model_state_dict)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    model.eval()

    model_wrapper = ModelWrapper(model)
    ig = IntegratedGradients(model_wrapper)
    # Compute the feature importance for this model
    for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in zip(loader_codes, loader_proteomics):
        inputs_codes, labels_codes = inputs_codes.to(device), labels_codes.to(device)
        inputs_proteomics, labels_proteomics = inputs_proteomics.to(device), labels_proteomics.to(device)
        
        def forward_func(proteomics, codes, lengths):
            return model(codes, proteomics, lengths)
        
        # Compute feature importances using a custom forward function; CONSIDER THE BASELINE
        importance_proteomics = ig.attribute(inputs_proteomics, additional_forward_args=(inputs_codes, lengths_codes))
        # Store the feature importance for this model
        importance_proteomics_all[model_idx] = importance_proteomics.mean(dim=0).cpu().detach()
    
# Compute the average feature importance across all models
importance_proteomics_avg = torch.mean((importance_proteomics_all), dim=0)*1e11

# Create dataframe for feature importances
importance_df_omics = pd.DataFrame([importance_proteomics_avg.numpy(), proteomics.drop(['sample_ID'],axis=1).columns]).T
importance_df_omics.columns = ['importance_omics','name']
importance_df_omics.sort_values('importance_omics')

### Plot Losses by Epoch

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

experiments = ['exp3', 'exp6']
colors = ['blue', 'orange']
mean_colors = ['black', 'red']

plt.figure(figsize=(10, 6))

max_epoch = max(max(len(model_data[10]) for model_data in results['mortality'][exp]) for exp in experiments) - 5

for experiment, color, mean_color in zip(experiments, colors, mean_colors):
    train_losses_all = []
    test_losses_all = []
    
    for model_data in results['mortality'][experiment]:
        train_losses = model_data[10][:-5]  # Exclude last 5 epochs
        test_losses = model_data[11][:-5]  # Exclude last 5 epochs

        train_losses_all.append(train_losses + [np.nan]*(max_epoch-len(train_losses)))
        test_losses_all.append(test_losses + [np.nan]*(max_epoch-len(test_losses)))

        # Plot each model's losses
        plt.plot(train_losses, test_losses, marker='o', linestyle='-', color=color, alpha=0.1)
    
    # Calculate the mean across each epoch for train and test losses separately
    train_losses_mean = np.ma.masked_invalid(train_losses_all).mean(axis=0)
    test_losses_mean = np.ma.masked_invalid(test_losses_all).mean(axis=0)

    # Sort by training loss
    sorted_indices = np.argsort(train_losses_mean)

    # Only plot the mean losses when we have at least 5 data points
    mask = np.count_nonzero(~np.isnan(train_losses_all), axis=0) >= 5

    # Plot the mean losses with the specific color
    plt.plot(train_losses_mean[sorted_indices][mask[sorted_indices]], 
             test_losses_mean[sorted_indices][mask[sorted_indices]], 
             color=mean_color, linewidth=2.0, label=f"{experiment} mean", zorder=100)

# Create custom patches for the legend
patch1 = mpatches.Patch(color='blue', label='Not PT models')
patch2 = mpatches.Patch(color='orange', label='PT models')
patch3 = mpatches.Patch(color='black', label='Not PT mean')
patch4 = mpatches.Patch(color='red', label='PT mean')

plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.xlabel('Training loss', fontsize=18)
plt.ylabel('Test loss', fontsize=18)
plt.title('Training losses vs Test losses for different models (log scale)',fontsize=18)
plt.legend(handles=[patch1, patch2, patch3, patch4])
plt.savefig('./losses.png', dpi=1000)


### Parameter Trajectory: Proteomics

In [ ]:
#return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = joint_model(RNN_data_codes.shape[2], best_hidden_dim, [best_hidden_dim], best_num_layers, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_PT_FT_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

all_outputs = []
model_numbers = []
epochs = []
PT = []

for i in tqdm(model_paths):
    result = re.search('both_PT_FT_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))

    with torch.no_grad():
        try:
            model_state_dict = torch.load(i)
            model.load_state_dict(model_state_dict)
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                inputs_proteomics = inputs_proteomics.to(device)
                outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][1]
            all_outputs.append(outputs.squeeze().cpu().numpy())
            model_numbers.append(model_number)
            epochs.append(epoch_number)
            PT.append('PT')
        except FileNotFoundError:
            pass

model = joint_model(RNN_data_codes_omics.shape[2], best_hidden_dim_omics, [best_hidden_dim_omics], best_num_layers_omics, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout_omics).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes_omics]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes_omics).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths_omics, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_omics_only_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

for i in tqdm(model_paths):
    result = re.search('both_omics_only_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))
    if True:
        with torch.no_grad():
            try:
                model_state_dict = torch.load(i)
                model.load_state_dict(model_state_dict)
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    inputs_proteomics = inputs_proteomics.to(device)
                    outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][1]
                all_outputs.append(outputs.squeeze().cpu().numpy())
                model_numbers.append(model_number)
                epochs.append(epoch_number)
                PT.append('NPT')
            except FileNotFoundError:
                pass
epochs = np.array(epochs)

In [ ]:
# Create custom lines for the legend
line1 = Line2D([0], [0], color='none', marker='o', markersize=10, markerfacecolor='darkred', label='PT')
line2 = Line2D([0], [0], color='none', marker='x', markersize=10, markerfacecolor='darkblue', label='NPT')

# Create a t-SNE instance and fit_transform the data
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create a markers array based on PT array
marker_map = {'PT': 'o', 'NPT': 'x'}
markers = [marker_map[pt] for pt in PT]

# Don't normalize the epochs
colors = epochs

# Create colormaps that goes from light red to dark red for PT, light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

# Plot the results with different markers and colors
for marker_type, marker in marker_map.items():
    mask = np.array(markers) == marker
    cmap = cmap_pt if marker_type == 'PT' else cmap_npt
    sc = plt.scatter(embedding[mask, 0], embedding[mask, 1], marker=marker, c=colors[mask], cmap=cmap, alpha = 0.7)

plt.gca().set_aspect('equal', 'datalim')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.title('Protein parameter space', fontsize=14)
plt.legend(handles=[line1, line2])  # add the custom legend
plt.savefig('./protein_params.png', dpi=1000)


In [ ]:
# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create colormaps that go from light red to dark red for PT, and light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

fig, axs = plt.subplots(5, 5, figsize=(20, 20))

# Calculate the global x and y limits
xlim = (embedding[:,0].min()-1, embedding[:,0].max()+1)
ylim = (embedding[:,1].min()-1, embedding[:,1].max()+1)

for model_number in range(25):
    ax = axs[model_number // 5, model_number % 5]

    for model_type in ['PT', 'NPT']:
        # Get mask for the current trajectory
        mask = (np.array(model_numbers) == model_number) & (np.array(PT) == model_type)

        # Get the points and corresponding epochs for the current trajectory
        trajectory_points = embedding[mask]
        trajectory_epochs = np.array(epochs)[mask]

        # Sort the points and epochs
        sort_indices = np.argsort(trajectory_epochs)
        sorted_points = trajectory_points[sort_indices]

        # Select the colormap based on model_type
        cmap = cmap_pt if model_type == 'PT' else cmap_npt

        # Plot the points with color indicating epoch and marker indicating PT/NPT
        ax.scatter(sorted_points[:, 0], sorted_points[:, 1], 
                   c=trajectory_epochs[sort_indices], 
                   cmap=cmap, 
                   marker=marker_map[model_type], 
                   alpha=0.6)

        # Plot lines connecting the points of the same model type
        ax.plot(sorted_points[:, 0], sorted_points[:, 1], 
                color='lightgrey' if model_type == 'PT' else 'black', 
                linestyle='--')

    ax.set_aspect('auto')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Turn off tick labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig('./protein_paths_cancer.png',dpi=1000)


In [ ]:
#return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = joint_model(RNN_data_codes.shape[2], best_hidden_dim, [best_hidden_dim], best_num_layers, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_PT_FT_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

all_outputs = []
model_numbers = []
epochs = []
PT = []

for i in tqdm(model_paths):
    result = re.search('both_PT_FT_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))

    with torch.no_grad():
        try:
            model_state_dict = torch.load(i)
            model.load_state_dict(model_state_dict)
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                inputs_proteomics = inputs_proteomics.to(device)
                outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][3]
            all_outputs.append(outputs.squeeze().cpu().numpy())
            model_numbers.append(model_number)
            epochs.append(epoch_number)
            PT.append('PT')
        except FileNotFoundError:
            pass

model = joint_model(RNN_data_codes_omics.shape[2], best_hidden_dim_omics, [best_hidden_dim_omics], best_num_layers_omics, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout_omics).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes_omics]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes_omics).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths_omics, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_omics_only_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

for i in tqdm(model_paths):
    result = re.search('both_omics_only_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))
    if True:
        with torch.no_grad():
            try:
                model_state_dict = torch.load(i)
                model.load_state_dict(model_state_dict)
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    inputs_proteomics = inputs_proteomics.to(device)
                    outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][3]
                all_outputs.append(outputs.squeeze().cpu().numpy())
                model_numbers.append(model_number)
                epochs.append(epoch_number)
                PT.append('NPT')
            except FileNotFoundError:
                pass
epochs = np.array(epochs)

In [ ]:
# Create custom lines for the legend
line1 = Line2D([0], [0], color='none', marker='o', markersize=10, markerfacecolor='darkred', label='PT')
line2 = Line2D([0], [0], color='none', marker='x', markersize=10, markerfacecolor='darkblue', label='NPT')

# Create a t-SNE instance and fit_transform the data
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create a markers array based on PT array
marker_map = {'PT': 'o', 'NPT': 'x'}
markers = [marker_map[pt] for pt in PT]

# Don't normalize the epochs
colors = epochs

# Create colormaps that goes from light red to dark red for PT, light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

# Plot the results with different markers and colors
for marker_type, marker in marker_map.items():
    mask = np.array(markers) == marker
    cmap = cmap_pt if marker_type == 'PT' else cmap_npt
    sc = plt.scatter(embedding[mask, 0], embedding[mask, 1], marker=marker, c=colors[mask], cmap=cmap, alpha = 0.7)

plt.gca().set_aspect('equal', 'datalim')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.title('Joint EHR-protein parameter space', fontsize=14)
plt.legend(handles=[line1, line2])  # add the custom legend
plt.savefig('./joint_params.png', dpi=1000)


In [ ]:
# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create colormaps that go from light red to dark red for PT, and light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

fig, axs = plt.subplots(5, 5, figsize=(20, 20))

# Calculate the global x and y limits
xlim = (embedding[:,0].min()-1, embedding[:,0].max()+1)
ylim = (embedding[:,1].min()-1, embedding[:,1].max()+1)

for model_number in range(25):
    ax = axs[model_number // 5, model_number % 5]

    for model_type in ['PT', 'NPT']:
        # Get mask for the current trajectory
        mask = (np.array(model_numbers) == model_number) & (np.array(PT) == model_type)

        # Get the points and corresponding epochs for the current trajectory
        trajectory_points = embedding[mask]
        trajectory_epochs = np.array(epochs)[mask]

        # Sort the points and epochs
        sort_indices = np.argsort(trajectory_epochs)
        sorted_points = trajectory_points[sort_indices]

        # Select the colormap based on model_type
        cmap = cmap_pt if model_type == 'PT' else cmap_npt

        # Plot the points with color indicating epoch and marker indicating PT/NPT
        ax.scatter(sorted_points[:, 0], sorted_points[:, 1], 
                   c=trajectory_epochs[sort_indices], 
                   cmap=cmap, 
                   marker=marker_map[model_type], 
                   alpha=0.6)

        # Plot lines connecting the points of the same model type
        ax.plot(sorted_points[:, 0], sorted_points[:, 1], 
                color='lightgrey' if model_type == 'PT' else 'black', 
                linestyle='--')

    ax.set_aspect('auto')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Turn off tick labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig('./joint_paths_cancer.png',dpi=1000)


In [ ]:
#return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = joint_model(RNN_data_codes.shape[2], best_hidden_dim, [best_hidden_dim], best_num_layers, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_PT_FT_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

all_outputs = []
model_numbers = []
epochs = []
PT = []

for i in tqdm(model_paths):
    result = re.search('both_PT_FT_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))

    with torch.no_grad():
        try:
            model_state_dict = torch.load(i)
            model.load_state_dict(model_state_dict)
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                inputs_proteomics = inputs_proteomics.to(device)
                outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][2]
            all_outputs.append(outputs.squeeze().cpu().numpy())
            model_numbers.append(model_number)
            epochs.append(epoch_number)
            PT.append('PT')
        except FileNotFoundError:
            pass

model = joint_model(RNN_data_codes_omics.shape[2], best_hidden_dim_omics, [best_hidden_dim_omics], best_num_layers_omics, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout_omics).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes_omics]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes_omics).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths_omics, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_omics_only_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

for i in tqdm(model_paths):
    result = re.search('both_omics_only_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))
    if True:
        with torch.no_grad():
            try:
                model_state_dict = torch.load(i)
                model.load_state_dict(model_state_dict)
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    inputs_proteomics = inputs_proteomics.to(device)
                    outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][2]
                all_outputs.append(outputs.squeeze().cpu().numpy())
                model_numbers.append(model_number)
                epochs.append(epoch_number)
                PT.append('NPT')
            except FileNotFoundError:
                pass
epochs = np.array(epochs)

In [ ]:
# Create custom lines for the legend
line1 = Line2D([0], [0], color='none', marker='o', markersize=10, markerfacecolor='darkred', label='PT')
line2 = Line2D([0], [0], color='none', marker='x', markersize=10, markerfacecolor='darkblue', label='NPT')

# Create a t-SNE instance and fit_transform the data
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create a markers array based on PT array
marker_map = {'PT': 'o', 'NPT': 'x'}
markers = [marker_map[pt] for pt in PT]

# Don't normalize the epochs
colors = epochs

# Create colormaps that goes from light red to dark red for PT, light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

# Plot the results with different markers and colors
for marker_type, marker in marker_map.items():
    mask = np.array(markers) == marker
    cmap = cmap_pt if marker_type == 'PT' else cmap_npt
    sc = plt.scatter(embedding[mask, 0], embedding[mask, 1], marker=marker, c=colors[mask], cmap=cmap, alpha = 0.7)

plt.gca().set_aspect('equal', 'datalim')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.title('EHR parameter space', fontsize=14)
plt.legend(handles=[line1, line2])  # add the custom legend
plt.savefig('./EHR_params.png', dpi=600)


In [ ]:
# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create colormaps that go from light red to dark red for PT, and light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

fig, axs = plt.subplots(5, 5, figsize=(20, 20))

# Calculate the global x and y limits
xlim = (embedding[:,0].min()-1, embedding[:,0].max()+1)
ylim = (embedding[:,1].min()-1, embedding[:,1].max()+1)

for model_number in range(25):
    ax = axs[model_number // 5, model_number % 5]

    for model_type in ['PT', 'NPT']:
        # Get mask for the current trajectory
        mask = (np.array(model_numbers) == model_number) & (np.array(PT) == model_type)

        # Get the points and corresponding epochs for the current trajectory
        trajectory_points = embedding[mask]
        trajectory_epochs = np.array(epochs)[mask]

        # Sort the points and epochs
        sort_indices = np.argsort(trajectory_epochs)
        sorted_points = trajectory_points[sort_indices]

        # Select the colormap based on model_type
        cmap = cmap_pt if model_type == 'PT' else cmap_npt

        # Plot the points with color indicating epoch and marker indicating PT/NPT
        ax.scatter(sorted_points[:, 0], sorted_points[:, 1], 
                   c=trajectory_epochs[sort_indices], 
                   cmap=cmap, 
                   marker=marker_map[model_type], 
                   alpha=0.6)

        # Plot lines connecting the points of the same model type
        ax.plot(sorted_points[:, 0], sorted_points[:, 1], 
                color='lightgrey' if model_type == 'PT' else 'black', 
                linestyle='--')

    ax.set_aspect('auto')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Turn off tick labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig('./EHR_paths_cancer.png',dpi=1000)


In [ ]:
#return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = joint_model(RNN_data_codes.shape[2], best_hidden_dim, [best_hidden_dim], best_num_layers, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_PT_FT_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

all_outputs = []
model_numbers = []
epochs = []
PT = []

for i in tqdm(model_paths):
    result = re.search('both_PT_FT_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))

    with torch.no_grad():
        try:
            model_state_dict = torch.load(i)
            model.load_state_dict(model_state_dict)
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                inputs_proteomics = inputs_proteomics.to(device)
                outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][4]
            all_outputs.append(outputs.squeeze().cpu().numpy())
            model_numbers.append(model_number)
            epochs.append(epoch_number)
            PT.append('PT')
        except FileNotFoundError:
            pass

model = joint_model(RNN_data_codes_omics.shape[2], best_hidden_dim_omics, [best_hidden_dim_omics], best_num_layers_omics, 1,
                    proteomics.shape[1]-1, [best_protein_hidden], [], best_dropout_omics).to(device)

model.eval()

criterion = nn.MSELoss()

input_proteomics = proteomics.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

scaler = StandardScaler()
input_proteomics = scaler.fit_transform(input_proteomics)

EHR_codes = [torch.tensor(data).float() for data in RNN_data_codes_omics]  
EHR_codes = [torch.nan_to_num(x) for x in EHR_codes]
input_proteomics = torch.tensor(input_proteomics).float()
input_proteomics = torch.nan_to_num(input_proteomics)
outcomes = torch.tensor(RNN_data_outcomes_omics).float()


data_set = DataBuilder(input_proteomics, outcomes, scaler)
loader_proteomics = DataLoader(dataset=data_set,batch_size=100000, worker_init_fn=worker_init_fn)
loader_codes = create_dataloaders(EHR_codes, outcomes, RNN_data_lengths_omics, 100000)
            

model_paths = [f'/mnt/project/cancer_mortality_3yr_LARGER_models_results/both_omics_only_{i}_epoch{j}.pth' for i in range(100) for j in range(200)]
model_number = 0

for i in tqdm(model_paths):
    result = re.search('both_omics_only_(\d+)_epoch(\d+).pth', i)
    model_number = int(result.group(1))
    epoch_number = int(result.group(2))
    if True:
        with torch.no_grad():
            try:
                model_state_dict = torch.load(i)
                model.load_state_dict(model_state_dict)
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(loader_codes, loader_proteomics)):
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    inputs_proteomics = inputs_proteomics.to(device)
                    outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)[1][4]
                all_outputs.append(outputs.squeeze().cpu().numpy())
                model_numbers.append(model_number)
                epochs.append(epoch_number)
                PT.append('NPT')
            except FileNotFoundError:
                pass
epochs = np.array(epochs)

In [ ]:
# Create custom lines for the legend
line1 = Line2D([0], [0], color='none', marker='o', markersize=10, markerfacecolor='darkred', label='PT')
line2 = Line2D([0], [0], color='none', marker='x', markersize=10, markerfacecolor='darkblue', label='NPT')

# Create a t-SNE instance and fit_transform the data
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto', perplexity=400)
embedding = tsne.fit_transform(np.array(all_outputs))

# Create a markers array based on PT array
marker_map = {'PT': 'o', 'NPT': 'x'}
markers = [marker_map[pt] for pt in PT]

# Don't normalize the epochs
colors = epochs

# Create colormaps that goes from light red to dark red for PT, light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

# Plot the results with different markers and colors
for marker_type, marker in marker_map.items():
    mask = np.array(markers) == marker
    cmap = cmap_pt if marker_type == 'PT' else cmap_npt
    sc = plt.scatter(embedding[mask, 0], embedding[mask, 1], marker=marker, c=colors[mask], cmap=cmap, alpha = 0.7)

plt.gca().set_aspect('equal', 'datalim')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.title('Overall parameter space', fontsize=14)
plt.legend(handles=[line1, line2])  # add the custom legend
plt.savefig('./overall_params.png', dpi=1000)


In [ ]:
# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
embedding = tsne.fit_transform(np.array(all_outputs))

# Create colormaps that go from light red to dark red for PT, and light blue to dark blue for NPT
cmap_pt = LinearSegmentedColormap.from_list("mycmap_pt", ["lightcoral", "darkred"])
cmap_npt = LinearSegmentedColormap.from_list("mycmap_npt", ["lightblue", "darkblue"])

fig, axs = plt.subplots(5, 5, figsize=(20, 20))

# Calculate the global x and y limits
xlim = (embedding[:,0].min()-1, embedding[:,0].max()+1)
ylim = (embedding[:,1].min()-1, embedding[:,1].max()+1)

for model_number in range(25):
    ax = axs[model_number // 5, model_number % 5]

    for model_type in ['PT', 'NPT']:
        # Get mask for the current trajectory
        mask = (np.array(model_numbers) == model_number) & (np.array(PT) == model_type)

        # Get the points and corresponding epochs for the current trajectory
        trajectory_points = embedding[mask]
        trajectory_epochs = np.array(epochs)[mask]

        # Sort the points and epochs
        sort_indices = np.argsort(trajectory_epochs)
        sorted_points = trajectory_points[sort_indices]

        # Select the colormap based on model_type
        cmap = cmap_pt if model_type == 'PT' else cmap_npt

        # Plot the points with color indicating epoch and marker indicating PT/NPT
        ax.scatter(sorted_points[:, 0], sorted_points[:, 1], 
                   c=trajectory_epochs[sort_indices], 
                   cmap=cmap, 
                   marker=marker_map[model_type], 
                   alpha=0.6)

        # Plot lines connecting the points of the same model type
        ax.plot(sorted_points[:, 0], sorted_points[:, 1], 
                color='lightgrey' if model_type == 'PT' else 'black', 
                linestyle='--')

    ax.set_aspect('auto')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Turn off tick labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig('./overall_paths_cancer.png',dpi=1000)
